# MNIST

In [1]:
import tensorflow as tf
import os

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")

from sklearn.preprocessing import StandardScaler

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["loss"], label="discriminative loss X", color="r")
    plt.plot(metrics["d_loss_Y"], label="discriminative loss Y", color="c")
    #plt.ylim(0,1)
    plt.legend()
    plt.savefig(os.path.join(path, "loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["acc"], label="generative loss X2Y", color="g")
    #plt.plot(metrics["g_loss_Y2X"], label="generative loss Y2X", color="b")
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(os.path.join(path, "acc" + str(epoch) + ".png"))
    plt.close()

In [3]:
# save metrics
def metrics_compare(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.rcParams["font.size"] = 25
    plt.yscale("log")
    plt.plot(metrics[0]["loss"], label="cnn", color="r")
    plt.plot(metrics[1]["loss"], label="cnn_BN", color="c")
    plt.plot(metrics[2]["loss"], label="SNN", color="b")
    #plt.plot(metrics[2]["loss"], label="discriminative loss Y", color="c")
    #plt.ylim(0,1)
    plt.xlabel("epoch", fontsize=25)
    plt.ylabel("loss", fontsize=25)
    plt.legend(fontsize=25)
    plt.savefig(os.path.join(path, "loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.rcParams["font.size"] = 25
    plt.plot(metrics[0]["acc"], label="cnn", color="r")
    plt.plot(metrics[1]["acc"], label="cnn_BN", color="c")
    plt.plot(metrics[2]["acc"], label="SNN", color="b")
    #plt.plot(metrics["g_loss_Y2X"], label="generative loss Y2X", color="b")
    plt.ylim(0,1)
    plt.xlabel("epoch", fontsize=25)
    plt.ylabel("acc", fontsize=25)
    plt.legend(fontsize=25)
    plt.savefig(os.path.join(path, "acc" + str(epoch) + ".png"))
    plt.close()

In [4]:
# plot images
def save_imgs(images, plot_dim=(1,2), size=(8,4), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    numImgs = images.shape[0]
    eachNumImgs = images.shape[0]/3

    #num_examples = plot_dim[0]*plot_dim[1]
    num_examples = numImgs
    size = (3*4, eachNumImgs*4)
    plot_dim = (eachNumImgs,3)

    fig = plt.figure(figsize=size)
    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((256, 256, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

# model

In [5]:
class CNN_only_Dropout:
    def __init__(self, name=""):
        self.reuse = False
        self.name = name
        print("init... name of d:" + self.name)
        
        # difinition of loss
        self.batch_size = 128

        self.epochs = 200
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParamter = 100
        self.metrics = {"loss":[], "acc":[]}

        img_size = 784
        num_class = 10
        # training data
        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size])
        self.y = tf.placeholder(tf.float32, shape=[None, num_class])
        
        # generate img from generated data
        out = self.model(self.X_tr) #X→Y→X

        lr = 0.025

        # the results of discrimination
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=self.y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(self.loss)
        
        # evaluate model
        correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(self.y,1 ))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.scaler = StandardScaler().fit(mnist.train.images)
        
    def model(self, x, df_dim=64):
        with tf.variable_scope('dropout' + self.name, reuse=self.reuse):
            # x: 28x28x1
            print("called... name of d:" + self.name)
            x = tf.reshape(x, shape=[-1, 28, 28, 1])
            conv1 = tf.layers.conv2d(x, 32, [5,5], [1,1], name="c1") # 28x28x32
            conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # 14x14
            
            conv2 = tf.layers.conv2d(x, 64, [4,4], [2,2], name="c2") # 14x14x64
            conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding="SAME") # 7x7x64

            shape = conv2.get_shape().as_list()
            dim = np.prod(shape[1:])
            
            conv2 = tf.reshape(conv2, [-1, dim])
            
            fc1 = tf.layers.dense(conv2, 1024, name='outputs')
            fc1 = tf.nn.dropout(fc1, 0.5)
            
            fc2 = tf.layers.dense(fc1, 10)
        return fc2
    
    def train(self):
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(self.epochs):
                batch_x, batch_y = mnist.train.next_batch(self.batch_size)
                batch_x_norm = self.scaler.transform(batch_x)
                _, loss, acc = sess.run([self.optimizer, self.loss, self.accuracy], feed_dict={self.X_tr: batch_x, self.y: batch_y})
                
                # 結果をappend
                self.metrics["loss"].append(loss)
                self.metrics["acc"].append(acc)
                
                print("epoch:" + str(epoch))

                # lossの可視化
                #if (epoch+1) % self.epoch_saveMetrics == 0:
                    #save_metrics(self.metrics, epoch)

                # parameterのsave
                if (epoch+1) % self.epoch_saveParamter == 0:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

In [6]:
class CNN_Relu:
    def __init__(self, batch_size, epochs, name=""):
        self.reuse = False
        self.name = name
        print("init... name of d:" + self.name)
        
        # difinition of loss
        self.batch_size = 128

        self.epochs = 200
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParamter = 100
        self.metrics = {"loss":[], "acc":[]}

        img_size = 784
        num_class = 10
        # training data
        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size])
        self.y = tf.placeholder(tf.float32, shape=[None, num_class])
        
        # generate img from generated data
        out = self.model(self.X_tr) #X→Y→X

        lr = 0.025

        # the results of discrimination
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=self.y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(self.loss)
        
        # evaluate model
        correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(self.y,1 ))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.scaler = StandardScaler().fit(mnist.train.images)
        
    def model(self, x, df_dim=64):
        with tf.variable_scope('Relu' + self.name, reuse=self.reuse):
            # x: 28x28x1
            print("called... name of d:" + self.name)
            x = tf.reshape(x, shape=[-1, 28, 28, 1])
            conv1 = tf.layers.conv2d(x, 32, [5,5], [1,1], name="c1") # 28x28x32
            conv1 = tf.nn.relu(conv1)
            conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # 14x14
            
            conv2 = tf.layers.conv2d(x, 64, [4,4], [2,2], name="c2") # 14x14x64
            conv2 = tf.nn.relu(conv2)
            conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding="SAME") # 7x7x64

            shape = conv2.get_shape().as_list()
            dim = np.prod(shape[1:])
            
            conv2 = tf.reshape(conv2, [-1, dim])
            
            fc1 = tf.layers.dense(conv2, 1024, name='outputs')
            #fc1 = tf.nn.dropout(fc1, 0.5)
            
            fc2 = tf.layers.dense(fc1, 10)
        return fc2
    
    def train(self):
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(self.epochs):
                batch_x, batch_y = mnist.train.next_batch(self.batch_size)
                batch_x_norm = self.scaler.transform(batch_x)
                _, loss, acc = sess.run([self.optimizer, self.loss, self.accuracy], feed_dict={self.X_tr: batch_x, self.y: batch_y})
                
                # 結果をappend
                self.metrics["loss"].append(loss)
                self.metrics["acc"].append(acc)
                
                #print("epoch:" + str(epoch))

                # lossの可視化
                #if (epoch+1) % self.epoch_saveMetrics == 0:
                    #save_metrics(self.metrics, epoch)

                # parameterのsave
                if (epoch+1) % self.epoch_saveParamter == 0:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

In [7]:
class CNN_BN:
    def __init__(self, batch_size, epochs, name=""):
        self.reuse = False
        self.name = name
        print("init... name of d:" + self.name)
        
        # difinition of loss
        self.batch_size = batch_size

        self.epochs = epochs
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParamter = 100
        self.metrics = {"loss":[], "acc":[]}

        img_size = 784
        num_class = 10
        # training data
        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size])
        self.y = tf.placeholder(tf.float32, shape=[None, num_class])
        
        # generate img from generated data
        out = self.model(self.X_tr) #X→Y→X

        lr = 0.025

        # the results of discrimination
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=self.y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(self.loss)
        
        # evaluate model
        correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(self.y,1 ))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.scaler = StandardScaler().fit(mnist.train.images)
        
    def model(self, x, df_dim=64):
        with tf.variable_scope('BN' + self.name, reuse=self.reuse):
            # x: 28x28x1
            print("called... name of d:" + self.name)
            x = tf.reshape(x, shape=[-1, 28, 28, 1])
            conv1 = tf.layers.conv2d(x, 32, [5,5], [1,1], name="c1") # 28x28x32
            conv1 = tf.layers.batch_normalization(conv1)
            conv1 = tf.nn.relu(conv1)
            conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # 14x14
            
            conv2 = tf.layers.conv2d(x, 64, [4,4], [2,2], name="c2") # 14x14x64
            conv2 = tf.layers.batch_normalization(conv2)
            conv2 = tf.nn.relu(conv2)
            conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding="SAME") # 7x7x64

            shape = conv2.get_shape().as_list()
            dim = np.prod(shape[1:])
            
            conv2 = tf.reshape(conv2, [-1, dim])
            
            fc1 = tf.layers.dense(conv2, 1024, name='outputs')
            #fc1 = tf.nn.dropout(fc1, 0.5)
            
            fc2 = tf.layers.dense(fc1, 10)
        return fc2
    
    def train(self):
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(self.epochs):
                batch_x, batch_y = mnist.train.next_batch(self.batch_size)
                batch_x_norm = self.scaler.transform(batch_x)
                _, loss, acc = sess.run([self.optimizer, self.loss, self.accuracy], feed_dict={self.X_tr: batch_x, self.y: batch_y})
                
                # 結果をappend
                self.metrics["loss"].append(loss)
                self.metrics["acc"].append(acc)
                
                #print("epoch:" + str(epoch))

                # lossの可視化
                #if (epoch+1) % self.epoch_saveMetrics == 0:
                    #save_metrics(self.metrics, epoch)

                # parameterのsave
                if (epoch+1) % self.epoch_saveParamter == 0:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

In [8]:
class SNN:
    def __init__(self, batch_size, epochs, name=""):
        self.reuse = False
        self.name = name
        print("init... name of d:" + self.name)
        
        # difinition of loss
        self.batch_size = batch_size

        self.epochs = epochs
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParamter = 100
        self.metrics = {"loss":[], "acc":[]}

        img_size = 784
        num_class = 10
        # training data
        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size])
        self.y = tf.placeholder(tf.float32, shape=[None, num_class])
        
        # generate img from generated data
        out = self.model(self.X_tr) #X→Y→X

        lr = 0.025

        # the results of discrimination
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=self.y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(self.loss)
        
        # evaluate model
        correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(self.y,1 ))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.scaler = StandardScaler().fit(mnist.train.images)
    
    def selu(self, x):
        #with ops.name_scope('elu') as scope:
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        return scale*tf.where(x>=0.0, x, alpha*tf.nn.elu(x))

    def model(self, x, df_dim=64):
        with tf.variable_scope('SNN' + self.name, reuse=self.reuse):
            # x: 28x28x1
            print("called... name of d:" + self.name)
            x = tf.reshape(x, shape=[-1, 28, 28, 1])
            conv1 = tf.layers.conv2d(x, 32, [5,5], [1,1], name="c1") # 28x28x32
            conv1 = self.selu(conv1)
            #conv1 = tf.layers.batch_normalization(conv1)
            #conv1 = tf.nn.relu(conv1)
            conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME") # 14x14
            
            conv2 = tf.layers.conv2d(x, 64, [4,4], [2,2], name="c2") # 14x14x64
            conv2 = self.selu(conv2)
            #conv2 = tf.layers.batch_normalization(conv2)
            #conv2 = tf.nn.relu(conv2)
            conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding="SAME") # 7x7x64

            shape = conv2.get_shape().as_list()
            dim = np.prod(shape[1:])
            
            conv2 = tf.reshape(conv2, [-1, dim])
            
            fc1 = tf.layers.dense(conv2, 1024, name='outputs')
            #fc1 = tf.nn.dropout(fc1, 0.5)
        
            fc2 = tf.layers.dense(fc1, 10)
        return fc2
    
    def train(self):
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(self.epochs):
                batch_x, batch_y = mnist.train.next_batch(self.batch_size)
                batch_x_norm = self.scaler.transform(batch_x)
                _, loss, acc = sess.run([self.optimizer, self.loss, self.accuracy], feed_dict={self.X_tr: batch_x, self.y: batch_y})
                
                # 結果をappend
                self.metrics["loss"].append(loss)
                self.metrics["acc"].append(acc)
                
                #print("epoch:" + str(epoch))

                # lossの可視化
                #if (epoch+1) % self.epoch_saveMetrics == 0:
                    #save_metrics(self.metrics, epoch)

                # parameterのsave
                if (epoch+1) % self.epoch_saveParamter == 0:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

# main func

In [9]:
if __name__ == "__main__":
    #cnn_only_dropout = CNN_only_Dropout()
    #cnn_only_dropout.train()
    
    cnn_relu = CNN_Relu(256, 2000)
    cnn_relu.train()
    
    cnn_BN = CNN_BN(256, 2000)
    cnn_BN.train()
    
    snn = SNN(256, 2000)
    snn.train()
    
    metrics_compare([cnn_relu.metrics, cnn_BN.metrics, snn.metrics])
    #cnn_selu = CNN_Selu()
    #cnn_selu.train()

init... name of d:
called... name of d:
init... name of d:
called... name of d:
init... name of d:
called... name of d:
